# Smart Query
This Notebook allows user to query a random piece of information from SumDB first. If the information is found in SumDB, it will then go to LogosCluster to find the full-length article. Otherwise, return nothing.

In [24]:
%pip install psycopg2 rich scikit-learn

from typing import List, Tuple
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from psycopg2 import sql
from rich import print
import psycopg2


Note: you may need to restart the kernel to use updated packages.


# HELPER FUNCTIONS

In [25]:
def log(message: str) -> None:
    """
    Logs a message to the console.
    """
    print(message)
    with open("sumdb_log.txt", "a") as log_file:
        log_file.write(message + "\n")

In [26]:
def get_column_names(conn, table):
    """
    Fetches and logs the column names of a specified table.
    """
    query = sql.SQL("""
        SELECT column_name 
        FROM information_schema.columns 
        WHERE table_name = %s
        ORDER BY ordinal_position;
    """)
    cur = conn.cursor()
    cur.execute(query, (table,))
    columns = cur.fetchall()
    cur.close()
    return [col[0] for col in columns]

# Perform SumDB Query

In [27]:
# sumb db config
sumdb_topic = "localhost"  # using localhost for now
port = "5432"
dbname = "db"  # internal database name
username = "user"
password = "password"

table = "test"  # Name of table to query

In [28]:
# Connect to the database
conn = psycopg2.connect(
    dbname=dbname,
    user=username,
    password=password,
    host=sumdb_topic,
    port=port
)

# Format datetime for readability
formatted_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
log(f"[{formatted_datetime}] Connected to database '{dbname}' on {sumdb_topic}:{port} as '{username}'")

# Get and log column names
column_names = get_column_names(conn, table)
log("Column names in '{}' table: {}".format(table, ", ".join(column_names)))


[2024-08-25 19:28:27] Connected to database 'db' on localhost:5432 as 'user'

Column names in 'test' table: id, chunkstart, chunkend, topic, summary, updatedat

In [29]:
def similarity_search(user_query: str, chunk_summary: str) -> float:
    """
    Calculate the similarity between the user query and a chunk summary using cosine similarity.
    """
    # Create a TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()

    # Combine the user query and chunk summary into a list
    documents = [user_query, chunk_summary]

    # Fit and transform the documents into TF-IDF matrix
    tfidf_matrix = vectorizer.fit_transform(documents)

    # Calculate the cosine similarity between the first and second document
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

    # Return the similarity score as a float
    return similarity[0][0]

In [30]:
def get_all_sumdb_data(conn: psycopg2.extensions.connection, table: str) -> List[Tuple[str]]:
    """
    Fetches all data from the specified table.
    """
    query = sql.SQL("SELECT id, chunkstart, chunkend, topic, summary, updatedat FROM {}").format(sql.Identifier(table))
    cur = conn.cursor()
    cur.execute(query)
    data = cur.fetchall()
    cur.close()
    return data

In [31]:
def max_relevant_score(chunk_summary: str, user_query: str) -> float:
    """
    Find the highest similarity score between the user query and the chunk summary.
    A chunk max relevant score is the highest similarity score between the user query and 1 line in the chunk summary.
    """
    
    # first format the chunk summary
    chunk_summary = chunk_summary.lower()
    
    # split the chunk summary into lines
    chunk_summary = chunk_summary.split("\n")
    
    # remove empty lines
    chunk_summary = [line for line in chunk_summary if line.strip()]
    
    # remove lines that are too short
    chunk_summary = [line for line in chunk_summary if len(line) > 5]
    
    # compare each line to the user query
    max_score = 0.0
    for line in chunk_summary:
        max_score = max(max_score, similarity_search(user_query, line))

    return max_score

In [32]:
def get_relevant_chunks(raw_data: List[Tuple[str]], user_query: str, threshold: float=0.5) -> List[Tuple[int, int, str, float]]:
    """
    Get a list of chunkstart Id values that are relevant to the user query.
    (which has similarity > threshold)
    """

    # Do similarity search for each chunk summary
    relevant_chunks = []
    
    max_score = 0.0
    # A chunk is considered relevant if 1 row within it is relevant
    for chunk_row in raw_data:
        chunk_start, chunk_end, topic, chunk_summary = chunk_row[:4]
        similarity = max_relevant_score(chunk_summary, user_query)
        if similarity > threshold:
            relevant_chunks.append((chunk_start, chunk_end, topic, similarity))

        max_score = max(max_score, similarity)
    print(f"Max score: {max_score}")
    return relevant_chunks

In [33]:
def find_detail_articles(logos_table: str, relevant_chunks: List[Tuple[int, int, str, float]]) -> List[Tuple[int, float]]:
    """
    Fetches the detail articles for the relevant chunks.
    """
    logos_dbname = "db"  # internal database name
    logos_username = "user"
    logos_password = "password"

    output = []
    for chunk in relevant_chunks:
        chunk_start, chunk_end, node_topic, _ = chunk
        
        # connect to db first
        logos_conn = psycopg2.connect(
            dbname=logos_dbname,
            user=logos_username,
            password=logos_password,
            host=node_topic,
        )
        
        # then do query on topic node
        query = sql.SQL("""
            SELECT question, answer, keywords FROM {} WHERE id BETWEEN %s AND %s
        """).format(sql.Identifier(logos_table))
        
        with logos_conn.cursor() as cur:
            cur.execute(query, (chunk_start, chunk_end))
            data = cur.fetchall()
            output.append(data)
            
        logos_conn.close()

    return output

In [34]:
log("Fetching all data from the database...")
raw_data = get_all_sumdb_data(conn, table)
log("Data fetched successfully.")

Fetching all data from the database...

Data fetched successfully.

In [35]:
user_query = r"Vertigo is a Marvel Comics character from the Savage Land with the power to induce severe dizziness and unconsciousness in others, first appearing in Marvel Fanfare #1 in 1982. Initially a member of the Marauders, she was involved in battles against the X-Men and has been killed and cloned multiple times, with varying portrayals in different media including animated series and video games."
threshold = 0.5
relevant_chunks = get_relevant_chunks(raw_data, user_query, threshold)

if not relevant_chunks:
    log("No relevant chunks found for query '{}'".format(user_query))
else:
    log("Relevant chunks for query '{}' are: {}".format(user_query, ", ".join(relevant_chunks)))

Max score: 0.0

No relevant chunks found for query 'Vertigo is a Marvel Comics character from the Savage Land with the power to 
induce severe dizziness and unconsciousness in others, first appearing in Marvel Fanfare #1 in 1982. Initially a 
member of the Marauders, she was involved in battles against the X-Men and has been killed and cloned multiple 
times, with varying portrayals in different media including animated series and video games.'

In [36]:
conn.close()